In [1]:
import carla, time, pygame, math, random, cv2
import numpy as np
from Utility import spawn_camera, speed_control, preprocess_image

pygame 2.6.1 (SDL 2.28.4, Python 3.7.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
client.load_world('Town05')
world = client.get_world()
spectator = world.get_spectator()
world.set_weather(carla.WeatherParameters.WetNight)

In [8]:
spawn_point = carla.Transform(carla.Location(-1, -21, 2), carla.Rotation(yaw=-90))

In [9]:
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
camera_retro = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=180, pitch=-40)))
camera_rigth = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-0.8, y=0.5, z=1.9), carla.Rotation(yaw=90, pitch=-40)))
run = True
video_output=None 

In [ ]:
vehicle.destroy()
camera_rigth.destroy()

True

In [10]:
def camera_rigth_callback(image):
    global video_output
    cv2.imwrite(f'./output3/{image.frame}.png', np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)))
    video_output = preprocess_image(np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)))
camera_rigth.listen(lambda image: camera_rigth_callback(image))

In [11]:
def angle_calculation(image):
    edges = cv2.Canny(image, 50, 150)
    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=100)
    min_angle = 75
    max_angle = 105
    min_angle_rad = np.deg2rad(min_angle)
    max_angle_rad = np.deg2rad(max_angle)
    theta_sum = 0
    line_count = 0
    for line in lines:
        rho, theta = line[0]
        if min_angle_rad <= theta <= max_angle_rad:
            theta_sum += theta
            line_count += 1
    if  line_count > 0:
        theta_mean = theta_sum / line_count
        theta_mean_deg = np.rad2deg(theta_mean) 
        if theta_mean_deg > 90:
            return theta_mean_deg+5
        if theta_mean_deg < 90:
            return theta_mean_deg-2
        return theta_mean_deg 
    return None

def mantieni_corsia(image, control, current_speed_mps, target_speed_mps):
    angolo = angle_calculation(image)
    control.steer = 0.0
    if angolo != None:
        valore_scalato = (((angolo - 75) / (105 - 75)) * 0.8) - 0.4
        control.steer = valore_scalato
    control = speed_control(control, target_speed_mps, current_speed_mps)
    return control


In [12]:
target_speed_mps = 10 / 3.6
control = carla.VehicleControl()
while run:
    current_velocity = vehicle.get_velocity()
    current_speed_mps = current_velocity.length()
    control = mantieni_corsia(video_output, control, current_speed_mps, target_speed_mps)
    vehicle.apply_control(control)
    time.sleep(0.05)

TypeError: 'NoneType' object is not iterable

In [82]:
vehicle.destroy()
camera_rigth.destroy()

True

In [1]:
import cv2
import numpy as np
import math

# Carica l'immagine in bianco e nero (grayscale)
image = cv2.imread('output/111931.png', cv2.IMREAD_GRAYSCALE)

# Applica il filtro di Canny per rilevare i bordi
edges = cv2.Canny(image, 50, 150)

# Crea una copia a colori per disegnare le linee rosse
color_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

# Applicare la trasformata di Hough per trovare le linee sugli edge
lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=100)

# Angolo desiderato in gradi (80° a 100°)
min_angle = 75
max_angle = 105

# Converte gli angoli in radianti
min_angle_rad = np.deg2rad(min_angle)
max_angle_rad = np.deg2rad(max_angle)
theta_sum = 0
line_count = 0

# Filtra le linee in base all'angolo
for line in lines:
    rho, theta = line[0]
    # Verifica se l'angolo è tra 80° e 100° (in radianti)
    if min_angle_rad <= theta <= max_angle_rad:
        # Calcolare le coordinate della linea
        theta_sum += theta
        line_count += 1
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        
        cv2.line(color_image, (x1, y1), (x2, y2), (0, 0, 255), 2)

theta_mean = theta_sum / line_count
theta_mean_deg = np.rad2deg(theta_mean)  # Converti in gradi
print(f'Media degli angoli (in gradi): {theta_mean_deg:.2f}')
# Mostra l'immagine con le linee rosse sugli edge
cv2.imshow('Lines between 80 and 100 degrees', color_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Media degli angoli (in gradi): 86.36
